# LangChain - Multiple retrievers

## Install dependencies

In [1]:
!pip install langchain
!pip install openai
!pip install pydantic
!pip install python-dotenv

## Load environment variables

Change the path if necessary (read `README.md` first and follow the instructions to setup the `.env` file):

In [2]:
import os
os.chdir("/home/jovyan/work/")

In [3]:
%load_ext dotenv
%dotenv

Disable warnings:

In [4]:
import warnings
warnings.filterwarnings('ignore')

## LangChain

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [6]:
model = OpenAI(temperature=0.0)

Format we want for the output:

In [7]:
class Movie(BaseModel):
    title: str = Field(description="the title of the movie")
    year: str = Field(description="year of the movie")
    director: str = Field(description="the name of the director of the movie")

Create a parser and inject the instructions from this parser into the prompt template:

In [8]:
parser = PydanticOutputParser(pydantic_object=Movie)

prompt = PromptTemplate(
    template="Recommend the most similar movie to this one (ignore sequels or prequels): {title}\n{format_instructions}\n",
    input_variables=["title"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

def get_similar_movie(title: str) -> Movie:
    _input = prompt.format_prompt(title=title)
    output = model(_input.to_string())
    return parser.parse(output)

In [9]:
get_similar_movie("Deep Impact")

Movie(title='Armageddon', year='1998', director='Michael Bay')

In [10]:
get_similar_movie("Matrix")

Movie(title='Inception', year='2010', director='Christopher Nolan')

In [11]:
get_similar_movie("Manhattan")

Movie(title='Annie Hall', year='1977', director='Woody Allen')

In [12]:
get_similar_movie("Avatar")

Movie(title='District 9', year='2009', director='Neill Blomkamp')

# Marvin

In [13]:
!pip install marvin

In [14]:
import datetime
from typing import Optional, List
from pydantic import BaseModel
from marvin import ai_model

In [15]:
class Client(BaseModel):
    name: str
    email: str
    phone: Optional[str]
    
class Issue(BaseModel):
    problem: str
    device: str

@ai_model
class Ticket(BaseModel):
    created: datetime.date
    client: Client
    title: str
    issues: List[Issue]

In [16]:
print(Ticket("""\
    Subject: Disappointment with My Gizmo 3000 and Widget Pro Devices (2023-07-15)

    Dear Gizmo and Widget Support Teams,

    I am disappointed with my recently purchased Gizmo 3000 and Widget Pro devices due
    to several issues.

    The Gizmo 3000's battery life is short, lasting only 4 hours, and the touchscreen
    is irresponsive. Meanwhile, the Widget Pro has constant connectivity issues and
    random shutdowns.

    I've contacted both support teams but have experienced long wait times and unhelpful
    representatives. I request prompt resolution, such as replacements, refunds, or
    effective troubleshooting.

    Sincerely,

    Jane Smith
    jane.smith@email.com
""").json(indent=2))

{
  "created": "2023-07-15",
  "client": {
    "name": "Jane Smith",
    "email": "jane.smith@email.com",
    "phone": ""
  },
  "title": "Disappointment with My Gizmo 3000 and Widget Pro Devices",
  "issues": [
    {
      "problem": "Short battery life and irresponsive touchscreen",
      "device": "Gizmo 3000"
    },
    {
      "problem": "Constant connectivity issues and random shutdowns",
      "device": "Widget Pro"
    }
  ]
}


# openai_function_calls

In [17]:
!pip install openai-function-call

In [18]:
from openai_function_call import OpenAISchema
from pydantic import Field
import openai

In [19]:
class Technology(BaseModel):
    name: str = Field(..., description="one specific technology (like a language, framework, platform)")
    category: str

class WorkExperience(BaseModel):
    role: str
    company: str
    start_date: str
    end_date: str
    responsibilities: List[str]

class Education(BaseModel):
    degree: str
    institution: str
    location: str
    start_date: str
    end_date: str

class Language(BaseModel):
    name: str
    level: str

class Resume(OpenAISchema):
    "Correctly extracted information from a resume"
    name: str
    contact_information: str
    objective_statement: str
    other_skills: List[str] = Field(..., description="name of a non-technical specific skill (like 'teamwork', 'empathy')")
    technologies: List[Technology]
    work_experience: List[WorkExperience]
    education: List[Education]
    certifications: List[str]
    language: List[Language]
    references: Optional[str]

In [20]:
def parse_resume(resume: str) -> Resume:
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        functions=[Resume.openai_schema],
        function_call={"name": Resume.openai_schema["name"]},
        messages=[
            {"role": "system", "content": "Extract all the information fot the resume provided"},
            {"role": "user", "content": resume},
        ],
    )
    return Resume.from_response(completion)

In [21]:
resume = """
    **John Doe**

    1234 Main Street, San Francisco, CA 94101 | (123) 456-7890 | john.doe@email.com | LinkedIn: /in/johndoe

    **Objective**

    Highly motivated Software Engineer specializing in Machine Learning, Artificial Intelligence, and MLOps, seeking to leverage extensive background in software development, data science, and AI systems in a challenging new role.

    **Skills**

    - Proficiency in Python, Java, and C++
    - Experience in Machine Learning and Deep Learning frameworks (TensorFlow, PyTorch, Keras)
    - Expertise in MLOps tools (Kubeflow, Docker, Jenkins)
    - Knowledge of cloud platforms (AWS, Google Cloud, Azure)
    - Experience with Big Data tools (Hadoop, Spark)
    - Excellent problem-solving skills and attention to detail

    **Work Experience**

    **Software Engineer, Machine Learning**
    Google, Mountain View, CA | June 2019 – Present

    - Designed and implemented ML models for various Google services, improving prediction accuracy by 20%.
    - Deployed robust ML pipelines using MLOps best practices.
    - Coordinated cross-functional teams to integrate ML models into product offerings.
    - Optimized machine learning algorithms to enhance system performance.

    **Software Engineer, AI**
    Microsoft, Redmond, WA | August 2016 – May 2019

    - Participated in the development of AI models for various Microsoft products.
    - Assisted in the establishment of the company's MLOps framework.
    - Created reusable code and libraries for future use.
    - Improved efficiency of existing systems by implementing AI solutions.

    **Junior Software Developer**
    IBM, Armonk, NY | July 2014 – July 2016

    - Developed and maintained software applications.
    - Collaborated with senior engineers to develop machine learning capabilities.
    - Assisted in bug fixing and improving application performance.

    **Education**

    **Master of Science in Computer Science (Specialization in AI)**
    Stanford University, Stanford, CA | 2012 – 2014

    **Bachelor of Science in Computer Science**
    University of California, Berkeley, CA | 2008 – 2012

    **Certifications**

    - Certified TensorFlow Developer – TensorFlow
    - Certified Professional in AI & ML – Microsoft
    - Certified Solutions Architect – Amazon Web Services

    **Languages**

    - English (native)
    - Spanish (fluent)
"""

In [22]:
print(parse_resume(resume).json(indent=2))

{
  "name": "John Doe",
  "contact_information": "1234 Main Street, San Francisco, CA 94101 | (123) 456-7890 | john.doe@email.com | LinkedIn: /in/johndoe",
  "objective_statement": "Highly motivated Software Engineer specializing in Machine Learning, Artificial Intelligence, and MLOps, seeking to leverage extensive background in software development, data science, and AI systems in a challenging new role.",
  "other_skills": [
    "Proficiency in Python, Java, and C++",
    "Experience in Machine Learning and Deep Learning frameworks (TensorFlow, PyTorch, Keras)",
    "Expertise in MLOps tools (Kubeflow, Docker, Jenkins)",
    "Knowledge of cloud platforms (AWS, Google Cloud, Azure)",
    "Experience with Big Data tools (Hadoop, Spark)",
    "Excellent problem-solving skills and attention to detail"
  ],
  "technologies": [
    {
      "name": "Python",
      "category": "Programming Languages"
    },
    {
      "name": "Java",
      "category": "Programming Languages"
    },
    {
  